In [4]:
from openai import OpenAI
from getpass import getpass     # 보안 정보를 화면에 표시 없이 입력받을 때 사용하는 모듈

In [20]:
MY_API_KEY = getpass('OpenAI API Key')

In [21]:
client = OpenAI(api_key=MY_API_KEY)

In [8]:
import json

In [9]:
data_path = 'FineTuneData/QRdataset.jsonl'

with open(data_path, 'r', encoding='utf-8') as f :
    dataset = [json.loads(line) for line in f]

In [10]:
from collections import defaultdict

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors['data_type'] += 1
        continue

    messages = ex.get('messages', None)
    if not messages:
        format_errors['missing_messages_list'] += 1
        continue

    for message in messages:
        if not isinstance(message, dict):  # message가 dict인지 확인
            format_errors['message_not_dict'] += 1
            continue

        if any(key not in ['role', 'content'] for key in message):  # 예상치 못한 키 확인
            format_errors['message_uncrecongnized_key'] += 1
            continue

        if 'role' not in message or 'content' not in message:
            format_errors['message_missing_key'] += 1

        if message.get('role', None) not in ['system', 'user', 'assistant']:
            format_errors['unrecognized_role'] += 1

        content = message.get('content', None)
        if not content or not isinstance(content, str):
            format_errors['missing_content'] += 1

    if not any(message.get('role', None) == 'assistant' for message in messages):
        format_errors['example_missing_assistant_message'] += 1

if format_errors:
    print('에러 발견')
    for key, value in format_errors.items():
        print(f'{key} : {value} 개')
else:
    print('에러가 없습니다.')


에러가 없습니다.


In [11]:
import tiktoken
import numpy as np

In [12]:
encoding = tiktoken.get_encoding('cl100k_base')

In [13]:
def num_tokens_from_messages(messages, tokens_per_message = 3, tokens_per_name = 1) :
    num_tokens = 0
    for message in messages :
        num_tokens += tokens_per_message
        for key, value in message.items() :
            num_tokens+= len(encoding.encode(value))
            if key == "name" :
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens


def num_assistant_tokens_from_messages(messages) :
    num_tokens = 0
    for message in messages :
        if message['role'] == 'assistant' :
            num_tokens += len(encoding.encode(message['content']))
    return num_tokens


def print_statistics(values) :
    print(f'min / max : {min(values)}, {max(values)}')
    print(f'mean / median : {np.mean(values)}, {np.median(values)}')

In [14]:
max_output_tokens = 4096       #gpt 3.5 turbo
n_missing_system = 0
n_missing_user = 0
n_messages = []
total_tokens_lens = []
assistant_message_lens = []

In [15]:
for ex in dataset :
    messages = ex['messages']

    if not any (message['role'] == 'system' for message in messages) :
        n_missing_system += 1

    if not any (message['role'] == 'user' for message in messages) :
        n_missing_user += 1

    n_messages.append(len(messages))

    total_tokens_lens.append(num_tokens_from_messages(messages))

    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))


print('System 메시지 누락 수 :', n_missing_system)
print('User 메시지 누락 수 :', n_missing_user)
print()
print('대화 당 메시지 수 통계 :')
print_statistics(n_messages)
print()
print('대화 당 전체 토큰 수 통계 :')
print_statistics(total_tokens_lens)
print()
print('대화 당 assistant 출력 토큰 수 통계 :')
print_statistics(assistant_message_lens)
print()
# total_tokens_lens에서 각각의 값을 16384와 비교
n_too_long = sum(i>max_output_tokens for i in total_tokens_lens)
print(f'{n_too_long}개의 대화가 {max_output_tokens}개의 토큰 제한을 초과하여 이 부분은 학습 중 잘릴 수 있습니다.')

System 메시지 누락 수 : 0
User 메시지 누락 수 : 0

대화 당 메시지 수 통계 :
min / max : 3, 3
mean / median : 3.0, 3.0

대화 당 전체 토큰 수 통계 :
min / max : 75, 127
mean / median : 87.51515151515152, 85.0

대화 당 assistant 출력 토큰 수 통계 :
min / max : 3, 4
mean / median : 3.5454545454545454, 4.0

0개의 대화가 4096개의 토큰 제한을 초과하여 이 부분은 학습 중 잘릴 수 있습니다.


In [16]:
MAX_TOKENS_PER_EXAMPLE = 4096      # 대화 당 최대 토큰 수 (3.5turbo 기준)

TARGET_EPOCHS = 3
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000

In [17]:
n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)

# 만약 대화의 개수와 TARGET_EPOCHS의 곱이 최소 데이터 수보다 적다면
if (n_train_examples * TARGET_EPOCHS) < MIN_TARGET_EXAMPLES :
    # OpenAI테스트 기준 epochs 최대치(MIN_DEFAULT_EPOCHS)와 최소 데이터에서 우리 데이터 수(n_train_examples)를 나눈 몫을 비교하여 더 낮은 값으로 최종 학습 횟수 설정
    n_epochs = min(MAX_DEFAULT_EPOCHS, (MIN_TARGET_EXAMPLES//n_train_examples))

elif (n_train_examples * TARGET_EPOCHS) > MAX_TARGET_EXAMPLES :
    n_epochs = max(MIN_DEFAULT_EPOCHS, (MAX_TARGET_EXAMPLES//n_train_examples))


n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in total_tokens_lens)

print(f'데이터 셋에는 학습 중 요금이 청구될 {n_billing_tokens_in_dataset}개의 토큰이 있습니다.')
print(f'기본적으로 이 데이터 셋에서 {n_epochs} epoch 동안 학습합니다.')
print(f'총 {n_epochs*n_billing_tokens_in_dataset}개의 토큰에 대해 요금이 청구됩니다.')

데이터 셋에는 학습 중 요금이 청구될 2888개의 토큰이 있습니다.
기본적으로 이 데이터 셋에서 3 epoch 동안 학습합니다.
총 8664개의 토큰에 대해 요금이 청구됩니다.


In [ ]:
fine_tune_files = client.files.create(
    file= open('FineTuneData/QRdataset.jsonl', 'rb'),
    purpose='fine-tune'
)

fine_tune_files

In [ ]:
fine_tune_job = client.fine_tuning.jobs.create(
    model = 'gpt-3.5-turbo',
    training_file=fine_tune_files.id
)

In [ ]:
fine_tune_job.id

In [ ]:
client.fine_tuning.jobs.retrieve(fine_tuning_job.id)

In [26]:
%%time
for _ in range(10) :
    completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                                messages=[{"role": "system", "content": "질문과 매물 추천으로 구분하여 각각 '질문', '추천'만을 출력합니다."},
                                                        {'role' : 'user', 'content' : '난 병원을 자주 가고 공원이 가까우면 좋겠어'}],
                                                temperature=0)

    print(completion.choices[0].message.content)

**추천**  
주변에 병원과 공원이 가까운 매물을 추천해 드릴게요.  
혹시 어떤 지역을 선호하시나요? 지역에 대한 선호도가 있으면 매물을 더 정확하게 추천해 드릴 수 있어요.
질문
추천
**추천**  
주변에 병원과 공원이 가까운 매물을 추천해 드릴게요.  
1. 매물 위치: 강남구 역삼동  
   - 병원: 강남성모병원 (도보 5분 이내)
   - 공원: 역삼동근린공원 (도보 10분 이내)
   - 매물 종류: 아파트  
   - 가격: 5억원  
   - 연락처: 010-1234-5678  

2. 매물 위치: 서초구 반포동  
   - 병원: 서울아산병원 (도보 3분 이내)
   - 공원: 반포한강공원 (도보 15분 이내)
   - 매물 종류: 주택  
   - 가격: 8억원  
   - 연락처: 010-9876-5432  

더 필요한 정보가 있으면 언제든지 말씀해주세요.
질문
**추천**  
주변에 병원과 공원이 가까운 지역을 찾으시는군요. 제가 추천하는 매물은 아래와 같습니다.  

매물 1: 서울 강남구 언주로에 위치한 아파트  
- 병원: 강남성모병원 (도보 5분 이내)
- 공원: 언주역 근처의 언주고개공원 (도보 10분 이내)
- 매물 가격: 5억원  

매물 2: 부산 해운대구 해운대로에 위치한 아파트  
- 병원: 부산해운대병원 (도보 10분 이내)
- 공원: 동백섬 공원 (도보 15분 이내)
- 매물 가격: 4억원  

매물 3: 대구 수성구 달구벌대로에 위치한 아파트  
- 병원: 대구파티마병원 (도보 5분 이내)
- 공원: 수성못 공원 (도보 10분 이내)
- 매물 가격: 3억원  

위 매물 중에서 원하시는 조건에 맞는 곳이 있으면 알려주세요. 더 많은 매물을 원하시면 다시 말씀해주세요.
**추천**  
주변에 병원과 공원이 가까운 매물을 찾아보시는 것이 좋겠네요. 아래 매물을 추천해드릴게요.  

매물 1: 서울 강남구 언주로에 위치한 아파트  
- 병원: 강남병원 도보 5분 거리  
- 공원: 언주 공원 인근에 위치  
- 매

In [27]:
%%time
for _ in range(10) :
    completion = client.chat.completions.create(
    model='ft:gpt-3.5-turbo-0125:fininsight::BByIChMO',
    messages=[
        {'role' : 'system', 'content' : '사용자의 입력을 질문과 매물 추천 요청으로 구분하여 각각 "질문", "추천"만을 출력합니다.'},
        {'role' : 'user', 'content' : '난 병원을 자주 가고 공원이 가까우면 좋겠어'}]
)

    print(completion.choices[0].message.content)

추천
추천
추천
추천
추천
추천
추천
추천
추천
추천
CPU times: total: 172 ms
Wall time: 8.31 s


In [30]:
%%time
for _ in range(10) :
    completion = client.chat.completions.create(
    model='ft:gpt-4o-mini-2024-07-18:fininsight::BBxvqdSu',
    messages=[
        {'role' : 'system', 'content' : '사용자의 입력을 질문과 매물 추천 요청으로 구분하여 각각 "질문", "추천"만을 출력합니다.'},
        {'role' : 'user', 'content' : '난 병원을 자주 가고 공원이 가까우면 좋겠어'}]
)

    print(completion.choices[0].message.content)

추천
추천
추천
추천
추천
추천
추천
추천
추천
추천
CPU times: total: 141 ms
Wall time: 12.2 s
